In [2]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.7 MB/s eta 0:00:00a 0:00:01


In [67]:
import csv
import requests
import psycopg2
import base64
import json

# Fakturoid API credentials and parameters
FAKTUROID_API_BASE_URL = 'https://app.fakturoid.cz/api/v2/accounts'
FAKTUROID_SLUG = 'datadrivenmarketingdev'
FAKTUROID_USERNAME = 'jan.kadlecek+dev@marketing.bi'
FAKTUROID_API_KEY = '31f47e8632bd4b8e77f3dc7b19c7e2c945c3108b'

# Form the complete URL with the slug
FAKTUROID_API_URL = f'{FAKTUROID_API_BASE_URL}/{FAKTUROID_SLUG}/invoices.json'

# PostgreSQL connection parameters
PG_PARAMS = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres",
    "port": "5432"
}

credentials = f"{FAKTUROID_USERNAME}:{FAKTUROID_API_KEY}"
credentials_encoded = base64.b64encode(credentials.encode()).decode('utf-8')
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {credentials_encoded}'
}

# Fetch invoices from Fakturoid API with slug parameter
response = requests.get(FAKTUROID_API_URL, headers=headers)
response.raise_for_status()
invoices = response.json()

In [68]:
print(invoices[0])

{'id': 23125376, 'custom_id': None, 'proforma': False, 'partial_proforma': False, 'number': '0021', 'number_format_id': 397206, 'variable_symbol': '21', 'your_name': 'Testovací účet', 'your_street': 'Testovací ', 'your_street2': None, 'your_city': 'Test', 'your_zip': '10000', 'your_country': 'CZ', 'your_registration_no': '', 'your_vat_no': '', 'your_local_vat_no': None, 'client_name': 'JVJ agency s.r.o.', 'client_street': 'Přibice 393', 'client_street2': None, 'client_city': 'Přibice', 'client_zip': '69124', 'client_country': 'CZ', 'client_registration_no': '28988566', 'client_vat_no': 'CZ28988566', 'client_local_vat_no': None, 'subject_id': 14300339, 'subject_custom_id': None, 'generator_id': None, 'related_id': None, 'correction': False, 'correction_id': None, 'paypal': False, 'gopay': False, 'token': 'IZDVlpInhw', 'status': 'overdue', 'order_number': '', 'issued_on': '2022-01-03', 'taxable_fulfillment_due': '2022-01-03', 'due': 14, 'due_on': '2022-01-17', 'sent_at': None, 'paid_at':

In [69]:
print(response.headers)

{'access-control-allow-origin': '*', 'cache-control': 'max-age=0, private, must-revalidate', 'content-encoding': 'gzip', 'content-type': 'application/json; charset=utf-8', 'etag': 'W/"97b81c46ef729b8d2de2647a1874fa31"', 'last-modified': 'Sun, 12 Feb 2023 23:38:43 GMT', 'referrer-policy': 'strict-origin-when-cross-origin', 'set-cookie': 'application_theme=eyJfcmFpbHMiOnsibWVzc2FnZSI6IklteHBaMmgwSWc9PSIsImV4cCI6IjIwMjQtMTItMDZUMTc6MTM6NTkuODYxWiIsInB1ciI6ImNvb2tpZS5hcHBsaWNhdGlvbl90aGVtZSJ9fQ%3D%3D--cfd36110ce7d603ba29d38f7414a85ad98a2558a; path=/; expires=Fri, 06 Dec 2024 17:13:59 GMT; HttpOnly; SameSite=Lax; secure', 'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'vary': 'Accept-Encoding', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-permitted-cross-domain-policies': 'none', 'x-request-id': '913246b6-70f1-4819-b718-5983572fe203', 'x-runtime': '0.040115', 'x-xss-protection': '0', 'date': 'Wed, 06 D

In [70]:
csv_file = 'invoices.csv'

# Write data to CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    headers = list(invoices[0].keys())  # Update headers
    csv_writer.writerow(headers)  # Write headers to the CSV file
    for invoice in invoices:
        # Extract values from the invoice dictionary based on the headers order
        data_row = [invoice[header] for header in headers]
        csv_writer.writerow(data_row)  # Write values to the CSV file

In [78]:
print(headers)

['id', 'custom_id', 'proforma', 'partial_proforma', 'number', 'number_format_id', 'variable_symbol', 'your_name', 'your_street', 'your_street2', 'your_city', 'your_zip', 'your_country', 'your_registration_no', 'your_vat_no', 'your_local_vat_no', 'client_name', 'client_street', 'client_street2', 'client_city', 'client_zip', 'client_country', 'client_registration_no', 'client_vat_no', 'client_local_vat_no', 'subject_id', 'subject_custom_id', 'generator_id', 'related_id', 'correction', 'correction_id', 'paypal', 'gopay', 'token', 'status', 'order_number', 'issued_on', 'taxable_fulfillment_due', 'due', 'due_on', 'sent_at', 'paid_at', 'reminder_sent_at', 'accepted_at', 'cancelled_at', 'webinvoice_seen_at', 'note', 'footer_note', 'private_note', 'tags', 'bank_account', 'iban', 'swift_bic', 'show_already_paid_note_in_pdf', 'payment_method', 'hide_bank_account', 'currency', 'exchange_rate', 'language', 'transferred_tax_liability', 'eu_electronic_service', 'oss', 'vat_price_mode', 'supply_code'

In [80]:
query = """
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'invoices';"""

# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        # Create table dynamically based on JSON keys
        table_creation_query = f"CREATE TABLE IF NOT EXISTS invoices ("
        for idx, header in enumerate(headers):
            # Define column names and types based on JSON keys
            value = invoices[0][header]
            if isinstance(value, (dict, list)): #TO DO: 
                column_type = "JSONB"  # Using JSONB for JSON-like structures
            elif "id" in header and isinstance(value, int):
                column_type = "INTEGER"
            elif isinstance(value, int) or str(value).isdigit():
                column_type = "REAL"  # Assuming other columns as VARCHAR for simplicity
            else:
                column_type = "VARCHAR(255)"  # Assuming other columns as VARCHAR for simplicity
            if idx != 0:
                table_creation_query += ", "
            table_creation_query += f"{header} {column_type}"
        table_creation_query += ");"

        cur.execute(table_creation_query)
        cur.execute(query)
        table_structure = cur.fetchall()

        print("Table Structure:")
        for column in table_structure:
            print(column)

Table Structure:
('paypal',)
('gopay',)
('partial_proforma',)
('client_registration_no',)
('number',)
('tags',)
('number_format_id',)
('subject_id',)
('due',)
('eet',)
('show_already_paid_note_in_pdf',)
('your_zip',)
('eet_records',)
('lines',)
('hide_bank_account',)
('variable_symbol',)
('proforma',)
('correction',)
('transferred_tax_liability',)
('eu_electronic_service',)
('client_zip',)
('id',)
('status',)
('order_number',)
('issued_on',)
('taxable_fulfillment_due',)
('due_on',)
('sent_at',)
('paid_at',)
('reminder_sent_at',)
('accepted_at',)
('cancelled_at',)
('webinvoice_seen_at',)
('note',)
('footer_note',)
('private_note',)
('bank_account',)
('iban',)
('swift_bic',)
('payment_method',)
('currency',)
('exchange_rate',)
('language',)
('oss',)
('vat_price_mode',)
('supply_code',)
('subtotal',)
('total',)
('native_subtotal',)
('native_total',)
('remaining_amount',)
('remaining_native_amount',)
('paid_amount',)
('eet_cash_register',)
('eet_store',)
('attachment',)
('html_url',)
('pub

In [81]:
# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        # Load data into PostgreSQL from CSV
        with open(csv_file, 'r', encoding='utf-8') as f:
    # Skip the header row in the CSV file
            next(f)
            cur.copy_from(f, 'invoices', sep=',', columns=headers)
            conn.commit()




BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY invoices, line 1: "23125376,,False,False,0021,397206,21,Testovací účet,Testovací ,,Test,10000,CZ,,,,JVJ agency s.r...."
